In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:13 http://security.ubuntu.com/ubuntu 

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-07-09 19:01:51--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.69MB/s    in 0.6s    

2022-07-09 19:01:52 (1.69 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Vine_Review").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Gift_Card_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Gift_Card_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   24371595|R27ZP1F1CD0C3Y|B004LLIL5A|     346014806|Amazon eGift Card...|       Gift Card|          5|            0|          0|   N|                Y|          Five Stars|Great birthday gi...| 2015-08-31|
|         US|   42489718| RJ7RSBCHUDNNE|B004LLIKVU|     473048287|Amazon.com eGift ...|       Gift Card|          5|    

In [5]:
# Create the vine_table. DataFrame
# vine_df = df.select([])

vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R27ZP1F1CD0C3Y|          5|            0|          0|   N|                Y|
| RJ7RSBCHUDNNE|          5|            0|          0|   N|                Y|
|R1HVYBSKLQJI5S|          5|            0|          0|   N|                Y|
|R2HAXF0IIYQBIR|          1|            0|          0|   N|                Y|
| RNYLPX611NB7Q|          5|            0|          0|   N|                Y|
|R3ALA9XXMBEDZR|          5|            0|          0|   N|                Y|
|R3R8PHAVJFTPDF|          5|            0|          0|   N|                Y|
|R18WWEK8OIXE30|          5|            0|          0|   N|                Y|
|R1EGUNQON2J277|          1|            0|          0|   N|                Y|
|R21Z4M4L98CPU2|          5|            0|          0|   N|     

In [6]:
# Filter the data to retrieve all the rows where the 'total_votes' count is equal to or greater than 20.
votes20_df = df.filter(df.total_votes >= 20)
votes20_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18888920|R36PQ9D8L2AAOH|B00JDQJVF2|     255898874|Amazon.com Gift C...|       Gift Card|          5|           30|         32|   N|                Y|          Five Stars|Love this adorabl...| 2015-08-26|
|         US|    4765280|R23GHBE86II0SK|B007V6ETDK|     924812503|Amazon eGift Card...|       Gift Card|          5|    

In [7]:
# Filter the new DataFrame to retrieve all the rows where the number of
# 'helpful_votes' divided by 'total_votes' is equal to or greater than 50%.

votes50_df = votes20_df.filter((votes20_df.helpful_votes/votes20_df.total_votes) >= 0.5)
votes50_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18888920|R36PQ9D8L2AAOH|B00JDQJVF2|     255898874|Amazon.com Gift C...|       Gift Card|          5|           30|         32|   N|                Y|          Five Stars|Love this adorabl...| 2015-08-26|
|         US|    4765280|R23GHBE86II0SK|B007V6ETDK|     924812503|Amazon eGift Card...|       Gift Card|          5|    

In [12]:
# Filter 'votes50_df' to retrieve all the rows where a review was written as part of the Vine program (paid), vine == 'Y'.
paid_df = votes50_df.filter(votes50_df.vine == 'Y')
paid_df.show()

+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+
|marketplace|customer_id|review_id|product_id|product_parent|product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|review_headline|review_body|review_date|
+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+
+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+



In [13]:
# Retrieve all the rows where the review was not part of the Vine program (unpaid), vine == 'N'.
unpaid_df = votes50_df.filter(votes50_df.vine == 'N')
unpaid_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18888920|R36PQ9D8L2AAOH|B00JDQJVF2|     255898874|Amazon.com Gift C...|       Gift Card|          5|           30|         32|   N|                Y|          Five Stars|Love this adorabl...| 2015-08-26|
|         US|    4765280|R23GHBE86II0SK|B007V6ETDK|     924812503|Amazon eGift Card...|       Gift Card|          5|    

In [15]:
# Total Paid Vine Reviews
paid_total = paid_df.count()
paid_total

0

In [17]:
# Total Unpaid Reviews
unpaid_total = unpaid_df.count()
unpaid_total

355

In [18]:
# 5 Star Paid Reviews
paid_five_star = paid_df.filter(paid_df.star_rating == 5).count()
paid_five_star

0

In [19]:
# 5 Star Unpaid Reviews
unpaid_five_star = unpaid_df.filter(unpaid_df.star_rating == 5).count()
unpaid_five_star

90

In [23]:
# 5 Star Paid Percenatge
paid_percentage = (paid_five_star/paid_total)*100
paid_percentage

# Since there are 0(ZERO) paid 5-star reviews, the percentage is 0%(ZERO%). CANNOT divide by zero to show this.

ZeroDivisionError: ignored

In [24]:
# 5 Star Unpaid Percenatge
unpaid_percentage = (unpaid_five_star/unpaid_total)*100
unpaid_percentage

25.352112676056336